In [124]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Add the correct relative path to the scripts folder
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src'))

In [14]:
df = pd.read_csv('aggregated_data.csv')

In [84]:
df

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
0,3.360100e+10,1.311448e+19,1.167200e+05,8.786906e+08
1,3.360100e+10,7.349883e+18,1.812300e+05,1.568596e+08
2,3.360100e+10,7.349883e+18,1.349690e+05,5.959665e+08
3,3.360101e+10,1.304243e+19,4.987800e+04,4.223207e+08
4,3.360101e+10,1.462771e+19,3.710400e+04,1.457411e+09
...,...,...,...,...
106850,3.378998e+10,2.615691e+19,2.103890e+05,1.094693e+09
106851,3.379000e+10,1.304243e+19,8.810000e+03,7.146416e+08
106852,3.379000e+10,1.311448e+19,1.409880e+05,4.803073e+08
106853,4.188282e+10,9.637368e+21,7.049061e+07,5.317447e+11


In [85]:
df_top =df.head(10)
df_top

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
0,3.360100e+10,1.311448e+19,116720.0,8.786906e+08
1,3.360100e+10,7.349883e+18,181230.0,1.568596e+08
2,3.360100e+10,7.349883e+18,134969.0,5.959665e+08
3,3.360101e+10,1.304243e+19,49878.0,4.223207e+08
4,3.360101e+10,1.462771e+19,37104.0,1.457411e+09
5,3.360101e+10,1.462771e+19,253983.0,6.152172e+08
6,3.360101e+10,1.462771e+19,128360.0,6.547231e+08
7,3.360101e+10,1.304243e+19,86399.0,3.326604e+08
8,3.360101e+10,1.469977e+19,495702.0,9.901322e+08
9,3.360102e+10,7.349883e+18,124854.0,7.324638e+08


In [86]:
df_least =df.tail(10)
df_least

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
106845,3.378994e+10,1.304243e+19,3.681200e+04,1.939172e+08
106846,3.378995e+10,2.622897e+19,2.090510e+05,1.540055e+09
106847,3.378996e+10,1.311448e+19,9.882600e+04,6.457777e+08
106848,3.378996e+10,1.311448e+19,1.570690e+05,7.102876e+08
106849,3.378997e+10,1.311448e+19,1.604610e+05,2.081231e+08
106850,3.378998e+10,2.615691e+19,2.103890e+05,1.094693e+09
106851,3.379000e+10,1.304243e+19,8.810000e+03,7.146416e+08
106852,3.379000e+10,1.311448e+19,1.409880e+05,4.803073e+08
106853,4.188282e+10,9.637368e+21,7.049061e+07,5.317447e+11
106854,3.197021e+12,7.277826e+18,2.415380e+05,2.321240e+08


In [87]:
data = pd.read_csv('aggregated_data_for_experiance.csv')

In [97]:
df.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total_Traffic
0,3.360100e+10,1.311448e+19,116720.0,8.786906e+08
1,3.360100e+10,7.349883e+18,181230.0,1.568596e+08
2,3.360100e+10,7.349883e+18,134969.0,5.959665e+08
3,3.360101e+10,1.304243e+19,49878.0,4.223207e+08
4,3.360101e+10,1.462771e+19,37104.0,1.457411e+09


In [100]:
# Selecting the experience metrics for clustering
X = df[['Bearer Id', 'Dur. (ms)', 'Total_Traffic']]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [101]:
# Apply K-means clustering (assuming 3 clusters for low, medium, high engagement)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(data)



KMeans(n_clusters=3, random_state=42)

In [117]:
# Applying K-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
data['engagment_cluster'] = kmeans.fit_predict(X_scaled)

# Display the clustering results
centroids_df =df[['Bearer Id', 'Dur. (ms)', 'Total_Traffic']]
centroids = kmeans.cluster_centers_
print("engagment_cluster:\n", centroids)


# Get the centroids of the clusters
#engagement_centroids = kmeans.cluster_centers_

# Convert centroids to DataFrame for analysis
#centroids_df = pd.DataFrame(engagement_centroids, columns=['Bearer Id', 'Dur. (ms)', 'Total_Traffic'])





engagment_cluster:
 [[ 3.01410884  4.2337798   0.38661062]
 [-0.1456524  -0.1200777  -0.4478551 ]
 [ 0.19207584 -0.06401364  1.71979473]]


In [118]:
# Identify the worst engagement cluster (users with lowest sessions, lowest duration)
worst_engagement_centroid = centroids_df.loc[centroids_df['Total_Traffic'].idxmin()]

In [ ]:
worst_engagement_centroid

In [120]:
# Function to calculate Euclidean distance (Engagement Score) for each user
def calculate_engagement_score(row, centroid):
    # Row values are number of sessions, pages per session, and session duration
    user_metrics = np.array([row['Bearer Id'], row['Dur. (ms)'], row['Total_Traffic']])
    
    # Calculate Euclidean distance
    distance = np.sqrt(np.sum((user_metrics - worst_engagement_centroid) ** 2))
    return distance

# Create DataFrame for engagement data (replace with actual columns)
df_engagement = pd.DataFrame(df, columns=['Bearer Id', 'Dur. (ms)', 'Total_Traffic'])

# Apply the function to each row to calculate the engagement score
df_engagement['Engagement_Score'] = df_engagement.apply(calculate_engagement_score, axis=1, centroid=centroids_df)

# Display the DataFrame with Engagement Score
print(df_engagement)

           Bearer Id     Dur. (ms)  Total_Traffic  Engagement_Score
0       1.311448e+19  1.167200e+05   8.786906e+08      5.836658e+18
1       7.349883e+18  1.812300e+05   1.568596e+08      7.205760e+16
2       7.349883e+18  1.349690e+05   5.959665e+08      7.205760e+16
3       1.304243e+19  4.987800e+04   4.223207e+08      5.764600e+18
4       1.462771e+19  3.710400e+04   1.457411e+09      7.349883e+18
...              ...           ...            ...               ...
106850  2.615691e+19  2.103890e+05   1.094693e+09      1.887908e+19
106851  1.304243e+19  8.810000e+03   7.146416e+08      5.764600e+18
106852  1.311448e+19  1.409880e+05   4.803073e+08      5.836658e+18
106853  9.637368e+21  7.049061e+07   5.317447e+11      9.630090e+21
106854  7.277826e+18  2.415380e+05   2.321240e+08      3.985640e+10

[106855 rows x 4 columns]


In [121]:
df_engagement

,Bearer Id,Dur. (ms),Total_Traffic,Engagement_Score
0,1.311448e+19,1.167200e+05,8.786906e+08,5.836658e+18
1,7.349883e+18,1.812300e+05,1.568596e+08,7.205760e+16
2,7.349883e+18,1.349690e+05,5.959665e+08,7.205760e+16
3,1.304243e+19,4.987800e+04,4.223207e+08,5.764600e+18
4,1.462771e+19,3.710400e+04,1.457411e+09,7.349883e+18
...,...,...,...,...
106850,2.615691e+19,2.103890e+05,1.094693e+09,1.887908e+19
106851,1.304243e+19,8.810000e+03,7.146416e+08,5.764600e+18
106852,1.311448e+19,1.409880e+05,4.803073e+08,5.836658e+18
106853,9.637368e+21,7.049061e+07,5.317447e+11,9.630090e+21


In [106]:
# Selecting the experience metrics for clustering
X = data[['TCP_Retransmission', 'RTT', 'Throughput']]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [107]:
# Applying K-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
data['Experience_Cluster'] = kmeans.fit_predict(X_scaled)

# Display the clustering results
print(data[['MSISDN/Number', 'TCP_Retransmission', 'RTT', 'Throughput', 'Experience_Cluster']])
centroids = kmeans.cluster_centers_
print("Cluster Centroids:\n", centroids)


        MSISDN/Number  TCP_Retransmission         RTT    Throughput  \
0        3.360100e+10        1.137460e+06   60.331977     76.000000   
1        3.360100e+10        1.137460e+06   31.000000     99.000000   
2        3.360100e+10        1.137460e+06  127.458589     97.000000   
3        3.360101e+10        2.132000e+03   84.000000    248.000000   
4        3.360101e+10        9.918360e+06   66.665988  25496.000000   
...               ...                 ...         ...           ...   
106850   3.378998e+10        5.688680e+05   98.229294  33937.500000   
106851   3.379000e+10        4.300880e+05   52.000000  10365.000000   
106852   3.379000e+10        1.137460e+06   40.000000    116.000000   
106853   4.188282e+10        6.222656e+06  152.671824  16253.501588   
106854   3.197021e+12        1.137460e+06  127.458589   1411.952463   

        Experience_Cluster  
0                        1  
1                        1  
2                        1  
3                        1  
4 

In [108]:
# Convert centroids to a DataFrame for easy analysis
centroids_df = pd.DataFrame(centroids, columns=['TCP_Retransmission', 'RTT', 'Throughput'])

# Identify the worst experience cluster based on the metrics (highest RTT and TCP retransmission)
worst_experience_centroid = centroids_df.loc[centroids_df['RTT'].idxmax()]
print("Worst Experience Centroid:\n", worst_experience_centroid)

Worst Experience Centroid:
 TCP_Retransmission    3.014109
RTT                   4.233780
Throughput            0.386611
Name: 0, dtype: float64


In [109]:
worst_experience_centroid = np.array(worst_experience_centroid)

# Calculate Euclidean distance (Experience Score) for each user
def calculate_experience_score(row, centroid):
    # Row values are TCP Retransmission, RTT, Throughput
    user_metrics = np.array([row['TCP_Retransmission'], row['RTT'], row['Throughput']])
    
    # Calculate Euclidean distance
    distance = np.sqrt(np.sum((user_metrics - centroid) ** 2))
    return distance

# Apply the function to each row to calculate the experience score
data['Experience_Score'] = data.apply(calculate_experience_score, axis=1, centroid=worst_experience_centroid)

# Display the updated DataFrame with the new Experience_Score column
experiance_score =data[['MSISDN/Number', 'TCP_Retransmission', 'RTT', 'Throughput', 'Experience_Score']]



In [110]:
experiance_score.head(10)

,MSISDN/Number,TCP_Retransmission,RTT,Throughput,Experience_Score
0,3.360100e+10,1137460.0,60.331977,76.0,1.137457e+06
1,3.360100e+10,1137460.0,31.000000,99.0,1.137457e+06
2,3.360100e+10,1137460.0,127.458589,97.0,1.137457e+06
3,3.360101e+10,2132.0,84.000000,248.0,2.144821e+03
4,3.360101e+10,9918360.0,66.665988,25496.0,9.918390e+06
5,3.360101e+10,571490.0,75.729294,3954.0,5.715007e+05
6,3.360101e+10,667008.0,33.665988,21256.5,6.673436e+05
7,3.360101e+10,556.0,52.000000,1247.0,1.364595e+03
8,3.360101e+10,1137460.0,127.458589,94.5,1.137457e+06
9,3.360102e+10,1137460.0,62.000000,146.0,1.137457e+06


In [122]:
# Combine the engagement and experience scores into one DataFrame
satisfaction_df = pd.concat([df_engagement, experiance_score], axis=1)

# Calculate the Satisfaction Score as the average of Engagement Score and Experience Score
satisfaction_df['Satisfaction_Score'] = (satisfaction_df['Engagement_Score'] + satisfaction_df['Experience_Score']) / 2

print("Satisfaction Scores:\n", satisfaction_df[['Engagement_Score', 'Experience_Score', 'Satisfaction_Score']])


Satisfaction Scores:
         Engagement_Score  Experience_Score  Satisfaction_Score
0           5.836658e+18      1.137457e+06        2.918329e+18
1           7.205760e+16      1.137457e+06        3.602880e+16
2           7.205760e+16      1.137457e+06        3.602880e+16
3           5.764600e+18      2.144821e+03        2.882300e+18
4           7.349883e+18      9.918390e+06        3.674942e+18
...                  ...               ...                 ...
106850      1.887908e+19      5.698764e+05        9.439542e+18
106851      5.764600e+18      4.302099e+05        2.882300e+18
106852      5.836658e+18      1.137457e+06        2.918329e+18
106853      9.630090e+21      6.222674e+06        4.815045e+21
106854      3.985640e+10      1.137458e+06        1.992877e+10

[106855 rows x 3 columns]


In [137]:
satisfaction_df['Satisfaction_Score']

0         2.918329e+18
1         3.602880e+16
2         3.602880e+16
3         2.882300e+18
4         3.674942e+18
              ...     
106850    9.439542e+18
106851    2.882300e+18
106852    2.918329e+18
106853    4.815045e+21
106854    1.992877e+10
Name: Satisfaction_Score, Length: 106855, dtype: float64

In [125]:
# Engagement and experience metrics combined as features (X) and satisfaction score as target (y)
X = np.hstack([df_engagement  , experiance_score  ])  # Combine engagement and experience metrics
y = satisfaction_df  # Satisfaction scores as target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict satisfaction scores on the test set
y_pred = model.predict(X_test)

# Display predicted satisfaction scores and actual satisfaction scores
print("Predicted Satisfaction Scores:", y_pred)
print("Actual Satisfaction Scores:", y_test)


Predicted Satisfaction Scores: [[1.46277089e+19 7.00489660e+04 7.16996950e+08 ... 5.02906866e+02
  1.13754135e+06 3.67494162e+18]
 [1.46277089e+19 2.48275528e+05 1.80675145e+09 ... 3.79729109e+02
  5.68900664e+05 3.67494162e+18]
 [1.31144836e+19 1.47012318e+05 1.17338117e+08 ... 1.05972595e+02
  1.13730103e+06 2.91832895e+18]
 ...
 [7.27782567e+18 2.61774234e+04 5.75084377e+08 ... 2.72054886e+04
  1.13904809e+06 6.59509372e+08]
 [7.34988325e+18 9.27052002e+04 6.48662225e+08 ... 5.73389331e+01
  1.13751457e+06 3.60287930e+16]
 [7.34988326e+18 1.26735575e+05 5.02829342e+08 ... 1.00165526e+02
  1.13746437e+06 3.60287995e+16]]
Actual Satisfaction Scores:            Bearer Id  Dur. (ms)  Total_Traffic  Engagement_Score  \
77690   1.462771e+19    70049.0   7.169970e+08      7.349883e+18   
81663   1.462771e+19   248277.0   1.806751e+09      7.349883e+18   
105055  1.311448e+19   147012.0   1.173381e+08      5.836658e+18   
61433   1.455565e+19   483076.0   1.381211e+09      7.277826e+18   
2

In [141]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate MSE and R²
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 1337847511602.107
R-squared: 0.9175529467139526
